In [1]:
from aocd import get_data

Given a 2D grid representing antennas with specific frequencies (letters, digits), identify locations called "antinodes" where signals resonate.

Rules for Antinodes:

**Same Frequency:** Two antennas must have the same frequency for there to be antinodes

**Collinearity:** The antennas and antinodes lie on a straight line

**Resonance Condition:** One antenna must be twice as far from the antinode as the other

**Antinode Count:** Each valid antenna pair generates two antinodes (one on each side)

**Objective:** Count the unique antinode locations within the grid bounds

In [2]:
example_input = """
............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............
"""

In [3]:
import numpy as np

In [4]:
example_grid = np.array([list(line) for line in example_input.strip().split("\n")])

In [5]:
example_grid

array([['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '0', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '0', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '0', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '0', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', 'A', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']],
      dtype='<U1')

In [18]:
example_grid.shape

(12, 12)

Identify the positions of all antennas with their corresponding "frequency" values

In [6]:
def parse_grid(grid):

    # boolean mask for alphanumeric elements
    mask = np.char.isalnum(grid)
    
    positions = np.argwhere(mask).astype(np.int64)
    antenna_values = grid[mask]

    # can't use np.column_stack because it would turn `positions` into strings to match `antenna_values` type
    # so here's a workaround
    antennas = np.empty((positions.shape[0], 3), dtype=object)
    antennas[:, :2] = positions
    antennas[:, 2] = antenna_values
    
    return antennas

In [7]:
antennas = parse_grid(example_grid); antennas

array([[1, 8, '0'],
       [2, 5, '0'],
       [3, 7, '0'],
       [4, 4, '0'],
       [5, 6, 'A'],
       [8, 8, 'A'],
       [9, 9, 'A']], dtype=object)

- Loop through all possible pairs of antennas that have the same frequency
- For each pair, find the potential antinode locations such that one antenna is twice as far from the antinode as the other
- Ensure that the calculated antinode locations are within the bounds of the map
- Use a `set` to store and deduplicate the valid antinode locations
- Return the number of unique locations stored in the set

### First Approach

In [8]:
def within_bounds(node, grid):
    return 0 <= node[0] < len(grid) and 0 <= node[1] < len(grid[0])

In [9]:
antinodes = set()
    
for i in range(len(antennas)):
    for j in range(i + 1, len(antennas)):
        x1, y1, val1 = antennas[i]
        x2, y2, val2 = antennas[j]
        
        if val1 != val2:
            continue
        
        # deltas for calculating antinode locations
        dx, dy = x2 - x1, y2 - y1
        
        # potential antinode locations
        antinode1 = (x1 - dx, y1 - dy)
        antinode2 = (x2 + dx, y2 + dy)
        
        # add valid antinodes that are within bounds
        if within_bounds(antinode1, example_grid):
            antinodes.add(antinode1)
        if within_bounds(antinode2, example_grid):
            antinodes.add(antinode2)

In [10]:
len(antinodes)

14

### Vecotorized

In [11]:
def find_antinodes(antennas, grid_shape):
    
    coords = antennas[:, :2]
    freq_values = antennas[:, -1]
    
    # create all unique pairs of indices (i, j) where i < j with this one weird trick
    i_indices, j_indices = np.triu_indices(len(antennas), k=1)
    
    coords1 = coords[i_indices]
    coords2 = coords[j_indices]
    freqs1 = freq_values[i_indices]
    freqs2 = freq_values[j_indices]
    
    matching_pairs = freqs1 == freqs2
    
    dx_dy = coords2[matching_pairs] - coords1[matching_pairs]
    
    # potential antinodes
    antinode1 = coords1[matching_pairs] - dx_dy
    antinode2 = coords2[matching_pairs] + dx_dy
    
    def within_bounds(coords, shape):
        return np.all((coords >= 0) & (coords < shape), axis=1)
    
    valid_antinode1 = antinode1[within_bounds(antinode1, grid_shape)]
    valid_antinode2 = antinode2[within_bounds(antinode2, grid_shape)]
    
    # combine and remove duplicates by converting to set of tuples
    antinodes = set(map(tuple, valid_antinode1)) | set(map(tuple, valid_antinode2))
    
    return antinodes


In [12]:
len(find_antinodes(antennas, example_grid.shape))

14

In [17]:
find_antinodes(parse_grid(example_grid), example_grid.shape)

{(0, 6),
 (0, 11),
 (1, 3),
 (2, 4),
 (2, 10),
 (3, 2),
 (4, 9),
 (5, 1),
 (5, 6),
 (6, 3),
 (7, 0),
 (7, 7),
 (10, 10),
 (11, 10)}

In [13]:
data = get_data(day=8, year=2024)

In [14]:
grid = np.array([list(line) for line in data.strip().split("\n")])

In [15]:
antennas = parse_grid(grid); antennas[:10]

array([[0, 21, '5'],
       [0, 37, 'P'],
       [1, 13, 'w'],
       [1, 19, 'T'],
       [1, 29, 'X'],
       [1, 30, 'h'],
       [1, 36, '5'],
       [1, 49, 'u'],
       [2, 19, 'k'],
       [2, 20, 'X']], dtype=object)

In [16]:
len(find_antinodes(antennas, grid.shape))

400

## Part Two

The extension in Part 2 introduces a new concept of *resonant harmonics*, which expands the way antinodes are determined compared to the original problem. 

An antinode occurs at any grid position exactly in line with at least two antennas of the same frequency, regardless of distance.
The three T-frequency antennas are all exactly in line with two antennas, so they are all also antinodes.

Antinodes are not only created by pairs of antennas that are in line with each other but also by the positions of the antennas themselves - if there is more than one antenna with the same frequency.

**Antennas can now be antinodes:** If an antenna shares its frequency with at least one other antenna, its position becomes an antinode.

**New antinode locations:** For each pair of antennas with the same frequency, antinodes will be generated not only between them (based on distance) but also at the antennas' positions if there are multiple antennas of the same frequency.

**Counting Unique Antinodes:** The goal is to count all unique positions that are antinodes, including those generated by overlapping antennas of the same frequency.

In [ ]:
new_example = """
T....#....
...T......
.T....#...
.........#
..#.......
..........
...#......
..........
....#.....
..........
"""

In [ ]:
new_grid = np.array([list(line) for line in new_example.strip().split("\n")])

In [ ]:
def find_antinodes(antennas, grid_shape):
    coords = antennas[:, :2]
    freq_values = antennas[:, -1]
    
    i_indices, j_indices = np.triu_indices(len(antennas), k=1)
    
    coords1 = coords[i_indices]
    coords2 = coords[j_indices]
    freqs1 = freq_values[i_indices]
    freqs2 = freq_values[j_indices]
    
    matching_pairs = freqs1 == freqs2
    
    dx_dy = coords2[matching_pairs] - coords1[matching_pairs]
    
    antinode1 = coords1[matching_pairs] - dx_dy
    antinode2 = coords2[matching_pairs] + dx_dy
    
    def within_bounds(coords, shape):
        return np.all((coords >= 0) & (coords < shape), axis=1)
    
    valid_antinode1 = antinode1[within_bounds(antinode1, grid_shape)]
    valid_antinode2 = antinode2[within_bounds(antinode2, grid_shape)]
    
    # Combine potential antinodes
    antinodes = set(map(tuple, valid_antinode1)) | set(map(tuple, valid_antinode2))

    # Identify antennas that share frequencies with at least one other antenna
    unique_freqs, counts = np.unique(freq_values, return_counts=True)
    shared_freqs = unique_freqs[counts > 1]

    antennas_in_pairs = set(map(tuple, coords[np.isin(freq_values, shared_freqs)]))

    # Combine calculated antinodes with antennas that share frequencies
    total_antinodes = antinodes | antennas_in_pairs

    return total_antinodes

In [ ]:
antennas = parse_grid(new_grid)
find_antinodes(antennas, new_grid.shape)

In [ ]:
len(find_antinodes(antennas, new_grid.shape))

In [ ]:
antennas = parse_grid(example_grid)
len(find_antinodes(antennas, example_grid.shape))

The answer should be 34.... tbh I don't really understand the task